In [26]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class MACD_RSI_Stoch_Strategy(Strategy):
    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low

        # Compute MACD, RSI and Stochastic Oscillator
        self.macd = self.I(ta.macd, pd.Series(close), fast=12, slow=26, signal=9)
        self.rsi = self.I(ta.rsi, pd.Series(close), length=14)
        stoch_k, stoch_d = self.I(ta.stoch, pd.Series(high), pd.Series(low), pd.Series(close), k=14, d=3)
        # stoch_k, stoch_d = self.I(ta.stoch, high, low, close, k=14, d=3)

        self.stoch_k = np.concatenate((np.full(14, np.nan), stoch_k))
        self.stock_d = np.concatenate((np.full(14, np.nan), stoch_d))

        # self.stoch = self.I(ta.stoch, high, low, close, length=14)

    def next(self):
        # If not in the market and MACD > 0, RSI < 30 and Stoch < 20, then Buy
        if not self.position and self.macd[-1] > 0 and self.rsi[-1] < 30 and self.stoch.k[-1] < 20:
            self.buy()

        # If in the market and MACD < 0 or RSI > 70 or Stoch > 80, then Sell
        elif self.position and (self.macd[-1] < 0 or self.rsi[-1] > 70 or self.stoch.k[-1] > 80):
            self.sell()

# Load your data
df = pd.read_csv('data/^ixic-60d.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
df.set_index('Datetime', inplace=True)

# Initialize Backtest
bt = Backtest(df, MACD_RSI_Stoch_Strategy, cash=1000000, commission=.002)

# Run Backtest
results = bt.run()

# Plot the results
bt.plot()

ValueError: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (1551,); indicator "stoch(14,3)"shape: (2, 1538), returned value: [[        nan         nan 73.15660828 ... 63.85929142 55.20579173
  50.86060577]
 [        nan         nan         nan ... 72.49539906 64.11844542
  56.64189631]])